Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [78]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
from tqdm import tqdm_notebook, tnrange
import random

'0.11.0rc2'

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (187361, 784) (187361, 10)
Validation set (8900, 784) (8900, 10)
Test set (9738, 784) (9738, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [9]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [13]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 18.303099
Training accuracy: 7.1%
Validation accuracy: 10.6%
Loss at step 100: 2.380372
Training accuracy: 72.0%
Validation accuracy: 70.9%
Loss at step 200: 1.891251
Training accuracy: 74.6%
Validation accuracy: 73.2%
Loss at step 300: 1.632010
Training accuracy: 76.2%
Validation accuracy: 74.1%
Loss at step 400: 1.462605
Training accuracy: 77.0%
Validation accuracy: 74.5%
Loss at step 500: 1.339857
Training accuracy: 77.8%
Validation accuracy: 74.8%
Loss at step 600: 1.244420
Training accuracy: 78.3%
Validation accuracy: 74.9%
Loss at step 700: 1.167014
Training accuracy: 78.7%
Validation accuracy: 75.2%
Loss at step 800: 1.102619
Training accuracy: 79.2%
Validation accuracy: 75.3%
Test accuracy: 82.9%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [14]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [17]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 16.205093
Minibatch accuracy: 10.2%
Validation accuracy: 13.5%
Minibatch loss at step 500: 1.634712
Minibatch accuracy: 76.6%
Validation accuracy: 75.9%
Minibatch loss at step 1000: 1.560967
Minibatch accuracy: 74.2%
Validation accuracy: 76.1%
Minibatch loss at step 1500: 1.057541
Minibatch accuracy: 79.7%
Validation accuracy: 77.0%
Minibatch loss at step 2000: 0.932328
Minibatch accuracy: 80.5%
Validation accuracy: 77.6%
Minibatch loss at step 2500: 1.025495
Minibatch accuracy: 72.7%
Validation accuracy: 78.4%
Minibatch loss at step 3000: 1.168621
Minibatch accuracy: 78.1%
Validation accuracy: 78.0%
Test accuracy: 85.2%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [2]:
pickle_ = 'sanitized.pickle'

with open(pickle_, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['sanitized_train']
  train_labels = save['san_train_labels']
  valid_dataset = save['cross_sanitized_valid']
  valid_labels = save['cross_san_valid_labels']
  test_dataset = save['cross_sanitized_test']
  test_labels = save['cross_san_test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (187361, 28, 28) (187361,)
Validation set (8900, 28, 28) (8900,)
Test set (9738, 28, 28) (9738,)


In [5]:
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (187361, 784) (187361, 10)
Validation set (8900, 784) (8900, 10)
Test set (9738, 784) (9738, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [5]:
accuracy(train_dataset[:1000, :], train_labels[:1000])

2.3999999999999999

In [62]:
subset = 2000
learning_rate = 0.5

graph = tf.Graph()
with graph.as_default():
    ### Tensors
    # Constants
    train_tensor = tf.constant(train_dataset[:subset, :])
    train_lab_tensor = tf.constant(train_labels[:subset])
    validation_tensor = tf.constant(valid_dataset)
    valid_lab_tensor = tf.constant(valid_labels)
        
    # Tensors that are going to be calibrated
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    
    biases = tf.Variable(tf.zeros([num_labels]))
    
    ### Operations
    logits = tf.matmul(train_tensor, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, train_lab_tensor)
    )
    
    ### Optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    ### More Operations
    # These are merely to report progress
    train_probabilities = tf.nn.softmax(logits)
    validation_logits = tf.matmul(validation_tensor, weights) + biases
    validation_probabilities = tf.nn.softmax(validation_logits)
        

In [63]:
steps = 2000

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    with tqdm_notebook(range(steps), desc='Running..') as bar:
        for step in bar:
            _, l, prob = session.run([optimizer, loss, train_probabilities])
            if not step % 100:
                tqdm.write('Loss at step %d: %f' % (step, l))
                tqdm.write('Training accuracy: %.1f%%' % accuracy(
                        prob, train_labels[:subset])
                          )
                tqdm.write('Validation accuracy: %.1f%%' % accuracy(
                        validation_probabilities.eval(), valid_labels))
                
        tqdm.write('Test accuracy: %.1f%%' % accuracy(validation_probabilities.eval(), valid_labels))
        

Loss at step 0: 18.450094
Training accuracy: 12.8%
Validation accuracy: 14.0%
Loss at step 100: 1.882128
Training accuracy: 74.5%
Validation accuracy: 67.6%
Loss at step 200: 1.304855
Training accuracy: 78.5%
Validation accuracy: 69.0%
Loss at step 300: 0.985207
Training accuracy: 81.6%
Validation accuracy: 69.4%
Loss at step 400: 0.776982
Training accuracy: 83.8%
Validation accuracy: 69.7%
Loss at step 500: 0.633967
Training accuracy: 86.7%
Validation accuracy: 69.9%
Loss at step 600: 0.529363
Training accuracy: 89.1%
Validation accuracy: 70.1%
Loss at step 700: 0.448411
Training accuracy: 91.0%
Validation accuracy: 70.1%
Loss at step 800: 0.383531
Training accuracy: 92.0%
Validation accuracy: 70.1%
Loss at step 900: 0.331070
Training accuracy: 93.2%
Validation accuracy: 70.0%
Loss at step 1000: 0.288507
Training accuracy: 94.0%
Validation accuracy: 70.1%
Loss at step 1100: 0.253692
Training accuracy: 94.9%
Validation accuracy: 70.2%
Loss at step 1200: 0.224931
Training accuracy: 95.6

In [68]:
batch_size = 256
learning_rate = 0.3

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
    )
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    #whole_dataset = tf.constant(train_dataset)
    #total_accuracy = tf.nn.softmax(tf.matmul(whole_dataset, weights) + biases)
        

In [69]:
steps = 30000
length = len(train_labels)

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    with tqdm_notebook(range(steps), desc='Optimizing..') as bar:
        for step in bar:
            offset = random.randint(1, length - batch_size)
            
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            
            feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
                        
            _, l, pred = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            
            if not step % 1000:
                tqdm.write("Minibatch loss at step %d: %f" % (step, l))
                tqdm.write("Minibatch accuracy: %.1f%%" % accuracy(pred,
                                                                   batch_labels))
                tqdm.write("Validation accuracy: %.1f%%" % accuracy(
                        valid_prediction.eval(), valid_labels))
                
        # tqdm.write("Train accuracy: %.1f%%" % accuracy(total_accuracy.eval(), train_labels))
       


Minibatch loss at step 0: 19.379684
Minibatch accuracy: 3.9%
Validation accuracy: 8.5%
Minibatch loss at step 1000: 1.432204
Minibatch accuracy: 75.8%
Validation accuracy: 74.1%
Minibatch loss at step 2000: 0.914650
Minibatch accuracy: 83.2%
Validation accuracy: 75.3%
Minibatch loss at step 3000: 0.946227
Minibatch accuracy: 78.5%
Validation accuracy: 76.1%
Minibatch loss at step 4000: 0.993977
Minibatch accuracy: 76.6%
Validation accuracy: 77.1%
Minibatch loss at step 5000: 1.060688
Minibatch accuracy: 76.6%
Validation accuracy: 77.2%
Minibatch loss at step 6000: 0.834163
Minibatch accuracy: 80.1%
Validation accuracy: 77.8%
Minibatch loss at step 7000: 0.743028
Minibatch accuracy: 80.1%
Validation accuracy: 77.9%
Minibatch loss at step 8000: 1.068367
Minibatch accuracy: 79.3%
Validation accuracy: 78.2%
Minibatch loss at step 9000: 0.767892
Minibatch accuracy: 79.7%
Validation accuracy: 78.9%
Minibatch loss at step 10000: 0.675482
Minibatch accuracy: 82.0%
Validation accuracy: 78.8%
M

In [73]:
batch = 128
hidden_nodes = 1024
learning_rate = 0.3

In [74]:
graph = tf.Graph()
with graph.as_default():
    # constants
    train_tensor = tf.placeholder(tf.float32, shape=(batch, image_size * image_size))
    label_tensor = tf.placeholder(tf.float32, shape=(batch, num_labels))
    valid_tensor = tf.constant(valid_dataset)
    valid_lab_tensor = tf.constant(valid_labels)
    
    # hidden layer
    hidden_weights = tf.Variable(tf.truncated_normal(shape=(image_size * image_size, hidden_nodes)))
    hidden_biases = tf.Variable(tf.zeros(hidden_nodes))
    hidden_layer = tf.nn.relu(tf.matmul(train_tensor, hidden_weights) + hidden_biases)
    
    # output layer
    weights = tf.Variable(tf.truncated_normal(shape=(hidden_nodes, num_labels)))
    biases = tf.Variable(tf.zeros(num_labels))
    logits = tf.matmul(hidden_layer, weights) + biases
    
    # output
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, label_tensor))
    
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    # progress tensors
    valid_relu = tf.nn.relu(tf.matmul(valid_tensor, hidden_weights) + hidden_biases)
    validation_prediction = tf.nn.softmax(tf.matmul(valid_relu, weights) + biases)
    train_prediction = tf.nn.softmax(logits)
    
                                  

In [83]:
steps = 1001
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    with tnrange(steps, desc='Optimizing..') as bar:
        for step in bar:
            offset = random.randint(1, length - batch)
            feed_data = train_dataset[offset: offset + batch]
            feed_labels = train_labels[offset: offset + batch]
            
            feed_dict = {train_tensor: feed_data, label_tensor: feed_labels}
            
            _, l, pred = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            

            if not step % 100:
                print('Minibatch loss: %f' % l)
                print('Minibatch accuraccy: %.1f%%' % accuracy(pred, feed_labels))
                print('Validation accuracy: %.1f%%' % accuracy(validation_prediction.eval(), valid_labels))

Minibatch loss: 319.170044
Minibatch accuraccy: 12.5%
Validation accuracy: 25.3%
Minibatch loss: 31.381210
Minibatch accuraccy: 75.0%
Validation accuracy: 76.2%
Minibatch loss: 46.387367
Minibatch accuraccy: 71.1%
Validation accuracy: 76.7%
Minibatch loss: 30.930502
Minibatch accuraccy: 80.5%
Validation accuracy: 76.7%
Minibatch loss: 37.441326
Minibatch accuraccy: 75.8%
Validation accuracy: 77.9%
Minibatch loss: 18.686893
Minibatch accuraccy: 78.1%
Validation accuracy: 80.2%
Minibatch loss: 11.944464
Minibatch accuraccy: 80.5%
Validation accuracy: 79.6%
Minibatch loss: 9.669384
Minibatch accuraccy: 84.4%
Validation accuracy: 80.9%
Minibatch loss: 13.537716
Minibatch accuraccy: 82.8%
Validation accuracy: 77.3%
Minibatch loss: 5.701038
Minibatch accuraccy: 83.6%
Validation accuracy: 81.2%
Minibatch loss: 8.222722
Minibatch accuraccy: 83.6%
Validation accuracy: 80.4%

